### This notebooks contains a peft training

In [9]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import (
    LoraConfig,
    get_peft_model,
)
import torch
from trl import SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
import unicodedata
import re
from tqdm import tqdm
import sacrebleu
import bitsandbytes as bnb


In [10]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [11]:
base_model = "meta-llama/Llama-3.2-1B"
new_model = "llama-3.2-1b-rup"

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

In [13]:

def apply_supplimentary_transformations(df_path):
    df = pd.read_csv(df_path)
    df_transformed = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
    # There are some words that have a -mi at the end, we will eliminate them also
    df_transformed.replace(r'\s*-\s*mi\b', '', regex=True, inplace=True)
    df_transformed.replace(r'\(i\)', 'i', regex=True, inplace=True)
    df_transformed.replace('γ', 'y', regex=True, inplace=True)
    df_transformed.replace(r'’', '', regex=True, inplace=True)
    df_transformed.replace(r'“', '', regex=True, inplace=True)
    df_transformed.replace(r'„', '', regex=True, inplace=True)
    df_transformed.columns = [str(q).strip() for q in df_transformed.columns]
    return df_transformed
# df_transformed.drop(columns=['ro', 'rup', 'translations'], inplace=True)
train_dataset = Dataset.from_pandas(apply_supplimentary_transformations("../dataset/nllb_corpus_train.csv"))
train_dataset = train_dataset.shuffle(seed=42)

test_dataset = Dataset.from_pandas(apply_supplimentary_transformations("../dataset/nllb_corpus_test.csv"))
test_dataset = test_dataset.shuffle(seed=42) 


def generate_prompt(data_point):
    begin_of_text = "<|begin_of_text|>"
    end_of_text = "<|end_of_text|>"
    start_header_id = "<|start_header_id|>"
    end_header_id = "<|end_header_id|>"
    eot_id = "<|eot_id|>"
    return {"text":
    f"""{begin_of_text}
        {start_header_id}system{end_header_id} Tradu urmatorul text din aromana in romana:
        {start_header_id}user{end_header_id} {data_point["rup"]}{eot_id}
        {start_header_id}assistant{end_header_id} {data_point["ro"]}{eot_id}
        {end_of_text}"""}

train_dataset = train_dataset.map(generate_prompt)
test_dataset = test_dataset.map(generate_prompt)

# dataset['text'][3]

C:\Users\gheto\AppData\Local\Temp\ipykernel_22080\786282711.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformed = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
C:\Users\gheto\AppData\Local\Temp\ipykernel_22080\786282711.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformed = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
Map: 100%|██████████| 3004/3004 [00:00<00:00, 29451.81 examples/s]


In [14]:

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: 
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [15]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [16]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
)

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 3004/3004 [00:00<00:00, 51091.77 examples/s]


In [18]:
trainer.train()

  1%|          | 100/13516 [00:29<59:04,  3.79it/s] 

{'loss': 3.111, 'grad_norm': 2.412323236465454, 'learning_rate': 0.00019866725899600178, 'epoch': 0.01}


  1%|▏         | 200/13516 [00:57<59:49,  3.71it/s]  

{'loss': 2.3463, 'grad_norm': 1.1858011484146118, 'learning_rate': 0.00019718643565822599, 'epoch': 0.01}


  2%|▏         | 300/13516 [01:24<59:08,  3.72it/s]  

...

100%|█████████▉| 13500/13516 [1:12:28<00:04,  3.70it/s]

{'loss': 1.9589, 'grad_norm': 1.4190388917922974, 'learning_rate': 2.3693173404412857e-07, 'epoch': 1.0}


c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\peft\utils\save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
100%|██████████| 13516/13516 [1:12:36<00:00,  3.71it/s]c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\peft\utils\save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
100%|██████████| 13516/13516 [1:12:38<00:00,  3.10it/s]

{'train_runtime': 4358.1605, 'train_samples_per_second': 6.203, 'train_steps_per_second': 3.101, 'train_loss': 2.0453084658927274, 'epoch': 1.0}


TrainOutput(global_step=13516, training_loss=2.0453084658927274, metrics={'train_runtime': 4358.1605, 'train_samples_per_second': 6.203, 'train_steps_per_second': 3.101, 'total_flos': 7099737032134656.0, 'train_loss': 2.0453084658927274, 'epoch': 0.9999630081751932})

In [ ]:
messages = [
    {
        "role": "user",
        "content": "kiro"
    }
]
begin_of_text = "<|begin_of_text|>"
end_of_text = "<|end_of_text|>"
start_header_id = "<|start_header_id|>"
end_header_id = "<|end_header_id|>"
eot_id = "<|eot_id|>"
prompt =f"""{begin_of_text}
{start_header_id}system{end_header_id} Tradu această propoziție din aromână în română.
{start_header_id}user{end_header_id} Te s-hiba, greaste tata-su al Teatire, - ficiorlu-a meu easte!{eot_id}
{start_header_id}assistant{end_header_id}"""

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

In [20]:
def translate(text, src_lang='rup', tgt_lang='ro', max_length=256):
    """Translate a text or list of texts"""
    if isinstance(text, str):
        text = [text]
    
    results = []
    for sentence in text:
        prompt = f"""{begin_of_text}
        {start_header_id}system{end_header_id} Tradu această propoziție din aromână în română.
        {start_header_id}user{end_header_id} {sentence}{eot_id}
        {start_header_id}assistant{end_header_id}"""
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.4, top_k=50, top_p=0.95)
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translation = result.split("assistant")[-1].strip()
        results.append(translation)
    
    return results

# Example usage
t = 'Ma, a lor la si paru c-amintara na vasilie-ntreaga'
print(translate(t, 'aromanian', 'romanian'))

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\generation\utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['A fost, de-aici, si paru ca au trecut inainte un bine intreaga - pe cineva \n        \n        \n        \n        \n        атися\n        \n        \n        аракт\n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        lásil\n        \n        \n        \n        ЎыџNЎыџN\n        \n        \n         ForCanBeConverted\n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n        \n         ForCanBeConvertedToF']


In [ ]:

df_ro_rup_test = pd.read_csv("../dataset/nllb_corpus_test.csv")
df_ro_rup_test['ro_pred'] = ''
df_ro_rup_test['rup_pred'] = ''

test_len = len(df_ro_rup_test)
test_len = 200
for i in tqdm(range(0, test_len)):
    rup_texts = df_ro_rup_test.loc[i, 'rup']
    
    if rup_texts:
        df_ro_rup_test.loc[i, 'ro_pred'] = translate(rup_texts, 'romanian', 'aromanian')



In [45]:
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF()
df_ro_rup_test_t = [re.sub(r'[\n:/]', '', el[0]) for el in df_ro_rup_test['ro_pred'].tolist() if el]

# df_ro_rup_test_t = [el[0] for el in df_ro_rup_test['ro_pred'].tolist() if el]

print("Aromanian to Romanian BLEU:", bleu_calc.corpus_score(df_ro_rup_test['ro'][:200].tolist(), [df_ro_rup_test_t][:200]))


Aromanian to Romanian BLEU: BLEU = 0.00 17.4/3.6/2.6/2.0 (BP = 0.000 ratio = 0.060 hyp_len = 478 ref_len = 8019)


In [46]:
df_ro_rup_test_v = []
for ref, pred in zip(df_ro_rup_test['ro'][:200].tolist(), df_ro_rup_test_t):
    if pred:
        df_ro_rup_test_v.append(ref)

print(len(df_ro_rup_test_v))
df_ro_rup_test_t = [el[:40] for el in df_ro_rup_test_t if el]
print(len(df_ro_rup_test_t))
print("Aromanian to Romanian BLEU:", bleu_calc.corpus_score(df_ro_rup_test_v, [df_ro_rup_test_t]))
print("Aromanian to Romanian CHRF:", chrf_calc.corpus_score(df_ro_rup_test_v, [df_ro_rup_test_t]))

200
200
Aromanian to Romanian BLEU: BLEU = 0.59 11.5/2.2/1.3/0.5 (BP = 0.292 ratio = 0.448 hyp_len = 478 ref_len = 1066)
Aromanian to Romanian CHRF: chrF2 = 8.83
